# Rinna NeoX SageMaker LoRA with CTranslate2

LoRA でファインチューニングした [Rinna NeoX](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft) を CTranslate2 で高速化し SageMaker でデプロイするサンプルコード。

前提条件：すでに [Rinna_Neox_LoRA_ja.ipynb](https://github.com/aws-samples/aws-ml-jp/blob/main/tasks/generative-ai/text-to-text/fine-tuning/instruction-tuning/Transformers/Rinna_Neox_LoRA_ja.ipynb) を実行しており LoRA ファイルがある想定。

検証は SageMaker Studio Notebook で ml.m5.2xlarge 上で PyTorch 2.0.0 Python 3.10 CPU Optimized コンテナで行いました。このノートブックは十分なメモリが必要なため ml.m5.2xlarge 以上のインスタンスタイプを推奨します。

In [ ]:
!pip install "sagemaker>=2.143.0" -U
!pip install "ctranslate2==3.16.0" "transformers==4.30.1" torch sentencepiece "protobuf==3.20" peft

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## Download and Extract Model

最後の「Rinna」のトレーニングジョブのアーティファクトを取得します。必要に応じて書き換えてください。

In [ ]:
import boto3
import sagemaker

def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

model_data = get_latest_training_job_artifact('Rinna')
    
!aws s3 cp {model_data} rinna.tar.gz

In [ ]:
!rm -rf /tmp/models/adapter && mkdir -p /tmp/models/adapter
!tar -xvf rinna.tar.gz -C /tmp/models/adapter --no-same-owner --wildcards adapter_*

## Merge and Save Model

CTranslate2 でまとめて変換するために LoRA をモデルにマージして保存します。

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

model_name = "rinna/japanese-gpt-neox-3.6b"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map={"": "cpu"}
)
model = PeftModel.from_pretrained(
    model,
    "/tmp/models/adapter",
    torch_dtype=torch.float16,
)
model = model.merge_and_unload()
model.save_pretrained(
    save_directory=f"/tmp/models/model",
    max_shard_size="400MB"  # モデルファイルを分割することでメモリ負荷を削減
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(
    save_directory=f"/tmp/models/model",
)

## Convert Model

モデルを CTranslate2 に最適化された形式に変換します。この処理はメモリを大きく利用するため十分なインスタンスサイズを選択してください。検証は m5.2xlarge で行いました。

In [ ]:
!rm -rf scripts/model
!ct2-transformers-converter --low_cpu_mem_usage --model /tmp/models/model --quantization int8 --output_dir scripts/model

In [ ]:
!ls -l scripts/

## Package and Upload Model

In [ ]:
!apt update -y
!apt install pigz -y

In [ ]:
%cd scripts
# !tar -czvf ../package.tar.gz *
!tar cv ./ | pigz -p 8 > ../package.tar.gz # 8 並列でアーカイブ
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"Rinna-LoRA-CTranslate2")
model_path

## Deploy Model

In [ ]:
from sagemaker.serializers import JSONSerializer

endpoint_name = "Rinna-LoRA-CTranslate"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="2.0",
    py_version='py310',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "tokenizer": "rinna/japanese-gpt-neox-3.6b",
            "model": "model",
            "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
            "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600"
    }
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
)

## Inference

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor_client=Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)
data = {
    "instruction": """ヴァージン・オーストラリアはいつから運航を開始したのですか？完結に答えてください。""".replace("\n", "<NL>"),  # システム
    "input": """ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・ブランドを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルーとして、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリアの破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。""".replace("\n", "<NL>"),  # ユーザー
    "max_new_tokens": 64,
    "sampling_temperature": 0.3,
    "stop_ids": [1, 0],
}
response = predictor_client.predict(
    data=data
)
print(response)

## Benchmark

689 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit response = predictor_client.predict(data=data)

## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()